# SAL Script - Slew and Track

Performs a single slew and track using SAL Script.  
It can be used as a minimum functionality prototype for the Soak Test.

## Notebook Setup

Start by importing relevat libraries, setting up domain, remotes, logs, etc.

In [18]:
%load_ext autoreload
%autoreload 2

import os
import sys
import asyncio
import logging
import yaml

import numpy as np

from astropy import units as u
from astropy.time import Time

from lsst.ts import salobj
from lsst.ts.idl.enums.Script import ScriptState
from lsst.ts.standardscripts.maintel.track_target import TrackTarget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
track_target = TrackTarget(index=1)
await track_target.start_task

In [6]:
mtcs = track_target.tcs

## Prepare Components

In [7]:
await mtcs.set_state(salobj.State.DISABLED, components=["mtmount", "mtrotator"])

In [8]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtmount", "mtrotator"])

In [9]:
az = mtcs.rem.mtmount.tel_azimuth.get().actualPosition
el = mtcs.rem.mtmount.tel_elevation.get().actualPosition

print(az, el)

2.0 62.0


In [10]:
await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=2, elevation=62)

In [11]:
await mtcs.rem.mtmount.cmd_homeBothAxes.start(timeout=300)

## Emulate ScriptQueue

In [14]:
radec = track_target.tcs.radec_from_azel(90., 60.)

In [15]:
configuration = yaml.safe_dump(
    {
        "slew_icrs": {
            "ra": float(radec.ra.hour),
            "dec": float(radec.dec.deg),
        }, 
        "rot_value": 0., 
        "rot_type": "Physical", 
        "track_for": 120., 
        "stop_when_done": True,
        "ignore": [
            "mtaos",
            "mtdome",
            "mtdometrajectory",
            "mthexapod_1",
            "mthexapod_2", 
            "mtm1m3", 
            "mtm2",
        ],
    }
)

print(configuration)

ignore:
- mtaos
- mtdome
- mtdometrajectory
- mthexapod_1
- mthexapod_2
- mtm1m3
- mtm2
rot_type: Physical
rot_value: 0.0
slew_icrs:
  dec: -25.96774747716869
  ra: 21.900966359881163
stop_when_done: true
track_for: 120.0



In [22]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
await track_target.set_state(ScriptState.UNCONFIGURED)

# Configure the script, which puts the ScriptState to CONFIGURED
config_data = track_target.cmd_configure.DataType()
config_data.config = configuration

await track_target.do_configure(config_data)

In [19]:
print("Start:", Time.now())
results = await track_target.run()
print("Done:", Time.now())

Start: 2022-11-17 20:36:13.053276


Done: 2022-11-17 20:38:55.881946
